# This code snippet is a Python script for real-time multi-object tracking using YOLOv5 for object detection and a Simple Online and Realtime Tracking (SORT) algorithm for tracking. Here's a breakdown of what the code does:


In [ ]:
#%pip install opencv-python
#%pip install torch
#%pip install pandas
#%pip install requests
#%pip install Pillow
#%pip install filterpy

import pandas as pd
import cv2
import numpy as np
import time
from filterpy.kalman import KalmanFilter
import time
from collections import defaultdict
import torch

# Load the YOLOv5 model (YOLOv5m is used for better accuracy)
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

# COCO class labels with custom names
class_labels = {
    2: 'Car',
    3: 'Motorbike',
    5: 'Bus',
    7: 'Truck',
    0: 'Person',
    1: 'Bicycle',
    9: 'Three-wheeler',  # Placeholder for custom class (if available)
}

# Dictionary to track object appearance times and first seen timestamps
track_times = {}
active_tracks = {}

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Function to log data to a text file
def log_to_file(object_id, label, duration, first_seen, last_seen):
    with open("tracking_log.txt", "a") as file:
        file.write(f"ID: {object_id}, Type: {label}, Duration: {duration:.2f} seconds, First seen: {first_seen}, Last seen: {last_seen}\n")

# SORT Tracker Class (Simple implementation for tracking)
class SORT:
    def __init__(self):
        self.trackers = []
        self.next_id = 0

    def update(self, detections):
        updated_tracks = []
        for det in detections:
            tracker = KalmanFilter(dim_x=7, dim_z=4)
            tracker.x[:4] = np.array([det[0], det[1], det[2], det[3]]).reshape((4, 1))
            tracker.F = np.eye(7)
            tracker.H = np.eye(4, 7)
            tracker.P *= 10.
            tracker.id = self.next_id
            self.next_id += 1
            updated_tracks.append(tracker)
        return updated_tracks

# Initialize SORT tracker
tracker = SORT()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally to correct axis
    frame = cv2.flip(frame, 1)

    # Object detection using YOLOv5
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()  # Bounding boxes (x1, y1, x2, y2, confidence, class)

    # Filter vehicle and person detections
    vehicle_classes = [2, 3, 5, 7, 1, 9]  # Car, Motorbike, Bus, Truck, Bicycle, Three-wheeler
    vehicles = [d for d in detections if int(d[5]) in vehicle_classes and d[4] > 0.4]
    people = [d for d in detections if int(d[5]) == 0 and d[4] > 0.4]  # Detect people (class 0)

    # Combine vehicles and people detections
    detections = vehicles + people

    # Track current frame's object IDs
    current_frame_ids = set()

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        label = class_labels.get(int(cls), 'Unknown')
        object_id = int(x1) + int(y1)  # Simple way to generate ID (replace with better tracking logic)

        current_frame_ids.add(object_id)

        # If the object is new, record the appearance time
        if object_id not in active_tracks:
            active_tracks[object_id] = {'first_seen': time.time(), 'label': label}

        # Draw bounding box in green
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
        # Display ID and label in red
        cv2.putText(frame, f'{label} ID: {object_id}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Check for objects that have left the frame
    for object_id in list(active_tracks.keys()):
        if object_id not in current_frame_ids:
            first_seen = active_tracks[object_id]['first_seen']
            last_seen = time.time()
            duration = last_seen - first_seen
            log_to_file(object_id, active_tracks[object_id]['label'], duration, first_seen, last_seen)  # Log data to file
            del active_tracks[object_id]  # Remove the object from active tracks

    # Display the frame with tracking
    cv2.imshow('Real-Time Multi-Object Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\Krithik R/.cache\torch\hub\ultralytics_yolov5_master


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
requirements:  AutoUpdate skipped (offline)


YOLOv5  2024-12-3 Python-3.12.7 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
C:\Users\Krithik R/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Krithik R/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Krithik R/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Krithik R/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.auto

# Now we included a phase where it records the objects log 